# TensorFlow Assignment: Reinforcement Learning (RL)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Kevin Liang

### Short answer

1\. One of the fundamental challenges of reinforcement learning is balancing *exploration* versus *exploitation*. What do these two terms mean, and why do they present a challenge?

`Exploration: Gathering more information about the environment; in other words, searching the policy space for potentially better (higher reward) solutions than what the agent currently knows.`

`Exploitation: Following the best policy already learned, with the goal of maximizing total reward.`

`The goal of the RL agent is to maximize the total reward it can receive throughout its lifetime. It may find certain solutions first, but these may be local optima--i.e. there may be better solutions out there. However, searching (exploring) for better solutions will necessitate trying out many bad trajectories, which decreases the agents total reward in the short term. Too much exploration, on the other hand, might mean the agent wastes actions on moves it believes have lower payoff.
In order to maximize the reward gained, it needs to also follow the policy it's learned (exploiting).`

2\. Another fundamental reinforcement learning challenge is what is known as the *credit assignment problem*, especially when rewards are sparse. 
What do we mean by the phrase, and why does this make learning especially difficult?
How does this interact with reward function design, where we have to be careful that our reward captures the true objective?

`If rewards are sparse or don't come until the end of an episode (e.g. winning or losing a game), then the agent will have had to navigate many states and actions before receiving any feedback. In such a situation, it is hard to tell which decisions led to final good (or bad) result. This makes learning difficult, as we can't properly assign credit for the reward to each decision.`

`For example, a chess bot can play like a grandmaster, but then make a single boneheaded move that leads to it being checkmated. Despite all the good moves it made before the checkmate, the entire trajectory is now credited with a loss.`

`However, designing a good reward function that provides good feedback frequently is often difficult or impossible, as it may interfere with the reward function capturing the true objective. As such, we often have to make a trade-off between the credit assignment problem and having an accurate reward function.`  

### Deep SARSA Cart Pole

[SARSA (state-action-reward-state-action)](https://en.wikipedia.org/wiki/State–action–reward–state–action) is another Q value algorithm that resembles Q-learning quite closely:

Q-learning update rule:
\begin{equation}
Q_\pi (s_t, a_t) \leftarrow (1 - \alpha) \cdot Q_\pi(s_t, a_t) + \alpha \cdot \big(r_t + \gamma \max_a Q_\pi(s_{t+1}, a)\big)
\end{equation}

SARSA update rule:
\begin{equation}
Q_\pi (s_t, a_t) \leftarrow (1 - \alpha) \cdot Q_\pi(s_t, a_t) + \alpha \cdot \big(r_t + \gamma Q_\pi(s_{t+1}, a_{t+1})\big)
\end{equation}

Unlike Q-learning, which is considered an *off-policy* network, SARSA is an *on-policy* algorithm. 
When Q-learning calculates the estimated future reward, it must "guess" the future, starting with the next action the agent will take. In Q-learning, we assume the agent will take the best possible action: $\max_a Q_\pi(s_{t+1}, a)$. SARSA, on the other hand, uses the action that was actually taken next in the episode we are learning from: $Q_\pi(s_{t+1}, a_{t+1})$. In other words, SARSA learns from the next action he actually took (on policy), as opposed to what the max possible Q value for the next state was (off policy).

Build an RL agent that uses SARSA to solve the Cart Pole problem. 

*Hint: You can and should reuse the Q-Learning agent we went over earlier. In fact, if you know what you're doing, it's possible to finish this assignment in about 30 seconds.*

In [1]:
# Based on: https://gym.openai.com/evaluations/eval_EIcM1ZBnQW2LBaFN6FY65g/

import random
import gym
import math
import numpy as np
import tensorflow as tf
from collections import deque

class DQNCartPoleSolver():
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.01, epsilon_log_decay=0.995, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=100000)
        self.env = gym.make('CartPole-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/cartpole-1', force=True)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init model
        self.state_ = tf.placeholder(tf.float32, shape=[None, 4])
        h = tf.layers.dense(self.state_, units=24, activation=tf.nn.tanh)
        h = tf.layers.dense(h, units=48, activation=tf.nn.tanh)
        self.Q = tf.layers.dense(h, units=2)
        
        self.Q_ = tf.placeholder(tf.float32, shape=[None, 2])
        loss = tf.losses.mean_squared_error(self.Q_, self.Q)
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        lr = tf.train.exponential_decay(0.01, self.global_step, 0.995, 1)
        self.train_step = tf.train.AdamOptimizer(lr).minimize(loss, global_step=self.global_step)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    ## CHANGES HERE (add next_action as a parameter)
    def remember(self, state, action, reward, next_state, next_action, done):
        # ... and make sure to append the next_action to our memory.
        self.memory.append((state, action, reward, next_state, next_action, done))

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.sess.run(self.Q, feed_dict={self.state_: state}))

    def get_epsilon(self, t):
        return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))

    def preprocess_state(self, state):
        return np.reshape(state, [1, 4])

    def replay(self, batch_size):
        x_batch, y_batch = [], []
        minibatch = random.sample(
            self.memory, min(len(self.memory), batch_size))
        
        ## CHANGES HERE: pull out next_action (along with other recorded data)
         # as we loop through the memories in our minibatch.
        for state, action, reward, next_state, next_action, done in minibatch:
            y_target = self.sess.run(self.Q, feed_dict={self.state_: state})
            ## CHANGES HERE: rather than take the maximum value in the Q-value vector,
            ## we pull out the value corresponding with the action we actually took,
            ## by indexing the vector at [next_action].
            y_target[0][action] = reward if done else reward + self.gamma * self.sess.run(self.Q, feed_dict={self.state_: next_state})[0][next_action]
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        self.sess.run(self.train_step, feed_dict={self.state_: np.array(x_batch), self.Q_: np.array(y_batch)})

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)

        for e in range(self.n_episodes):
            ## CHANGES HERE: We calculate an initial action
            ## outside the loop, so that as we play Cartpole,
            ## we are operating at a one-time-step "lag": we can then
            ## record (previous_state, previous_action, reward, new_state, new_action)
            state = self.preprocess_state(self.env.reset())
            action = self.choose_action(state, self.get_epsilon(e))
            done = False
            i = 0
            while not done:
                if e % 100 == 0 and not self.quiet:
                    self.env.render()
                next_state, reward, done, _ = self.env.step(action)
                next_state = self.preprocess_state(next_state)
                ## CHANGES HERE: compute and store the *next* action we will take,
                ## so that later, when replaying our memories, we will know not just
                ## the action we took in a given state, but also the next action.
                next_action = self.choose_action(next_state, self.get_epsilon(e))
                self.remember(state, action, reward, next_state, next_action, done)
                state = next_state
                ## CHANGES HERE: at the end of the loop, we set our last action to
                ## the next action we've computed, preparing us for the next iteration.
                action = next_action
                i += 1

            scores.append(i)
            mean_score = np.mean(scores)
            if mean_score >= self.n_win_ticks and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            if e % 100 == 0 and not self.quiet:
                print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))

            self.replay(self.batch_size)
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = DQNCartPoleSolver()
    agent.run()

C:\Users\kevin_000\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[Episode 0] - Mean survival time over last 100 episodes was 14.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 13.9 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 51.37 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 114.1 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 192.14 ticks.
Ran 404 episodes. Solved after 304 trials ✔


Note: you should be able to find that SARSA works much better for the demo we went over during lecture.
This is not necessarily a general result.
Q-learning and SARSA tend to do better on different kinds of problems.